In [1]:
'''
    总体要求: "获取人员信息，并存储"
'''
# 导入必备的包
import requests
from lxml import etree
import json
import pandas as pd
import os
import random
import time
from concurrent.futures import ThreadPoolExecutor
# 指定工作目录
os.chdir(r'F:\zhihu_multithreading')

In [3]:
# 设置爬取前的环境 包括 IP代理、请求头、url接口地址
global proxies_list
global header
global basic_info_all
basic_info_all = [] # 统一的容器装数据

proxies_list = ['101.37.79.125:3128','120.92.74.189:3128','120.92.74.237:3128','119.31.210.170:7777','125.62.26.197:3128']
header = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.92 Safari/537.36'
}
# 获取所有用户id的url接口
url_empid = r'http://it.bbdservice.com:8909/GiveLike/user/giveLike/getOrg.html'
# 获取所有用户信息的url母体接口
url_base = r'http://it.bbdservice.com:8909/GiveLike/user/giveLike/index.html?empId=%s'

# 获取所有用户id
def user_id_list(url_empid):
    user_id_lis = []
    html_response = requests.get(url_empid,proxies={'http:':random.choice(proxies_list)},headers=header)
    html_json = json.loads(html_response.text)
    for item in html_json['emp']:
        user_id_lis.append(item['id'])
    return user_id_lis

# 获取所有用户信息的url链接
def url_list(url_base):
    url_list = []
    user_id_lis = user_id_list(url_empid)
    for user_id in user_id_lis:
        url_list.append(url_base % str(user_id))
    return url_list

# print(url_list(url_base))

In [4]:
#提取需要爬取的信息函数
def data_tq(obj_html,path):
    data = []
    infos = obj_html.xpath(path)
    for info in infos:
        data.append(info.text)
    return data

# 将爬取的信息 封装成字典的形式
def transform_dict(lis):
    dic_info = {}   
    i = 0
    while i < len(lis):
        dic_key = lis[i].replace(':','')
        dic_info[dic_key] = lis[i+1]
        i += 2
    return dic_info
# print(basic_info_dic)

In [5]:
# 爬取函数
def spider_pach(url):
    # 返回响应的数据以HTML 文本展示 
    html_response = requests.get(url,proxies={'http:':random.choice(proxies_list)},headers=header)
    # 内容序列化
    obj_html = etree.HTML(html_response.text)
    # 获得爬取信息
    user_name = data_tq(obj_html,r'//div[@class="username"]/b')[0]
    print("user_name:",user_name)
    basic_info = data_tq(obj_html,r'//div[@class="dataZL_h"]//b')
    user_show = data_tq(obj_html,r'//div[@class="usershow"]//p')
    # 合并用户的基本数据
    basic_info.extend(user_show)
    # 转换成字典
    basic_info_dic = transform_dict(basic_info)
    # 添加用户姓名
    basic_info_dic['姓名'] = user_name
    basic_info_all.append(basic_info_dic) # 将数据装入容器
    
    return basic_info_all

# 爬取完成后的 回调函数
def spider_callback(future):
    '''
        存储数据
    '''
    dataFrame = pd.DataFrame(future.result())
    writer = pd.ExcelWriter('人员信息.xlsx')
    dataFrame.to_excel(writer,'sheet1',index=False)
    writer.save()
    return print('数据存储完成！')

In [6]:
'''
    多线程爬取
'''
pool = ThreadPoolExecutor(6) # 采用多线程的方式进行爬取,先定义一个线程池
url_lis = url_list(url_base) # url 的集合体
for url in url_lis:
    vv = pool.submit(spider_pach,url)  
    
vv.add_done_callback(spider_callback) # 采集爬取完成后执行回调函数
pool.shutdown(wait=True)  # 爬取完毕后关闭线程池

user_name: 杨玉玲
user_name: 夏阳
user_name: 金洁
user_name: 王丹
user_name: 曹先亚
user_name: 杨建蓉
user_name: 李冰玉
user_name: 张巧玲
user_name: 马金华
user_name: 徐文静
user_name: 蒲盈竹
user_name: 王莹腾
user_name: 程子涵
user_name: 姜珊珊
user_name: 刘毅
user_name: 周利敏
user_name: 邹聪
user_name: 张洪波
user_name: 但宇
user_name: 卢巧慧
user_name: 高誉
user_name: 李亚琼
user_name: 胡兵
user_name: 刘苗苗
user_name: 李俊兵
user_name: 朱强
user_name: 张祎迪
user_name: 林凡亮
user_name: 陈肯
user_name: 邹钦霖
user_name: 胡文翠
user_name: 郭跃春
user_name: 顾磊
user_name: 刘海洋
user_name: 詹永川
user_name: 吴师成
user_name: 方彦
user_name: 刘永永
user_name: 朱琨
user_name: 左璇
user_name: 周林
user_name: 王丽娟
user_name: 唐青池
user_name: 杨坤
user_name: 吴娇
user_name: 冯飞翔
user_name: 程兰
user_name: 杨梅
user_name: 张锟
user_name: 刘成芬
user_name: 王圣
user_name: 吕敏
user_name: 李倩
user_name: 饶茜琳
user_name: 张佳
user_name: 廖敏
user_name: 胥冲
user_name: 李婉玲
user_name: 杨强
user_name: 水明芝
user_name: 魏伊利
user_name: 蒲伟晴
user_name: 陈鑫
user_name: 周力
user_name: 马扬
user_name: 张嘉锷
user_name: 何杰
user_name: 孟科
user_name: 田

user_name: 蔡江瑶
user_name: 孙赫然
user_name: 严丽1
user_name: 严薇
user_name: 周定宁
user_name: 黄宏汇
user_name: 蔡庆松
user_name: 钟秋1
user_name: 沈伟奇
user_name: 张健
user_name: 袁先智
user_name: 何宏靖
user_name: 黄艳
user_name: 李国强
user_name: 景桂杰
user_name: 余杭
user_name: 何宇
user_name: 钟秋
user_name: 张剑锋
user_name: 宋振东
user_name: 王果
user_name: 罗震林
user_name: 许斌
user_name: 李梦君
user_name: 李龙飞
user_name: 范东来
user_name: 王瑞贤
user_name: 郑倩
user_name: 冉智伟
user_name: 廖仕伟
user_name: 刘畅
user_name: 冯军
user_name: 何彬彬
user_name: 鲍蓉
user_name: 谯明文
user_name: 周林1
user_name: 陈瑶茜
user_name: 金涛1
user_name: 李元豪
user_name: 杨春林
user_name: 韦耀
user_name: 朱俊燊
user_name: 李鑫
user_name: 钱锋
user_name: 文开俊
user_name: 周如意
user_name: 陈双冬雪
user_name: 王梦佳
user_name: 王璨
user_name: 李小峰
user_name: 张黔
user_name: 王帆
user_name: 李敏
user_name: 高嘉瑜
user_name: 李雨秦
user_name: 张波
user_name: 刘筱
user_name: 金潇
user_name: 许光超
user_name: 钱柔柔
user_name: 龚思化
user_name: 谭艳
user_name: 陈沿吉
user_name: 钟育方
数据存储完成！
